In [1]:
import os
from PIL import Image
import time

def compress_images_recursive(root_folder, max_size_mb=4):
    max_size = max_size_mb * 1024 * 1024  # 바이트로 변환

    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if not filename.lower().endswith(".jpg"):
                continue

            filepath = os.path.join(dirpath, filename)
            filesize = os.path.getsize(filepath)

            if filesize <= max_size:
                print(f"✅ {filename}: {filesize/1024:.1f}KB → 유지")
                continue

            try:
                img = Image.open(filepath).convert("RGB")  # ✅ RGB 강제 변환
                quality = 95
                step = 5
                compressed = False

                for _ in range(10):
                    temp_path = filepath + ".temp.jpg"
                    img.save(temp_path, format="JPEG", quality=quality, optimize=True)

                    time.sleep(0.1)  # 파일 저장 안정화
                    temp_size = os.path.getsize(temp_path)

                    if temp_size <= max_size:
                        img.close()
                        os.remove(filepath)  # 원본 삭제
                        os.rename(temp_path, filepath)  # ✅ 안전하게 교체
                        print(f"⚠️ {filename}: 압축됨 → {temp_size/1024:.1f}KB (quality={quality})")
                        compressed = True
                        break

                    quality -= step

                if not compressed:
                    img.close()
                    if os.path.exists(temp_path):
                        os.remove(temp_path)
                    print(f"❌ {filename}: 압축 실패 (최저 quality={quality + step})")

            except Exception as e:
                print(f"❌ {filename}: 처리 중 오류 발생 → {e}")



In [6]:
compress_images_recursive("0613ciga_test2")    # 바꾸고 싶은 폴더주소 입력

✅ 360_F_311285826_hpnEQj8txKtX5mCl4K7oHH4lkGuRxXvU.jpg: 86.5KB → 유지
✅ 360_F_473057378_dvyrCRb7ZlOy0E8witgFIF6fkYw8LI0r.jpg: 92.9KB → 유지
✅ 3df993938ec00a4fa2b5fc545e4d4a7b0975537f.71.9.9.3.jpg: 179.5KB → 유지
✅ 4429bdc8f4bed76d1d63907a2008bfbd-1536x864.jpg: 980.6KB → 유지
✅ drain5.jpg: 219.8KB → 유지
✅ imgdac05e8dzik0zj.jpg: 98.5KB → 유지
✅ o1080116614786240065.jpg: 369.1KB → 유지
✅ o2304129614627900845.jpg: 719.4KB → 유지
✅ OIP (1).jpg: 68.6KB → 유지
✅ OIP (2).jpg: 51.6KB → 유지
✅ OIP.jpg: 59.2KB → 유지
✅ R (1).jpg: 38.1KB → 유지
✅ R.jpg: 35.5KB → 유지
✅ tsdr-pl-pipe230618t-5.jpg: 503.4KB → 유지


### 점진적으로 4mb 이하로 줄여주는 코드

In [3]:
import os
from PIL import Image
from io import BytesIO

def compress_image_to_under_4mb(image_path, temp_path, max_bytes=4 * 1024 * 1024, initial_quality=95):
    img = Image.open(image_path).convert("RGB")
    quality = initial_quality
    resize_factor = 0.95

    while True:
        buffer = BytesIO()
        img.save(buffer, format="JPEG", quality=quality)
        size = buffer.tell()

        if size <= max_bytes:
            break

        w, h = img.size
        img = img.resize((int(w * resize_factor), int(h * resize_factor)), Image.LANCZOS)
        quality = max(quality - 5, 30)

        if quality <= 30 and min(img.size) < 300:
            print(f"⚠️ 너무 작아져서 압축 실패: {image_path}")
            return False

    # 안정적으로 임시 파일에 저장
    img.save(temp_path, format="JPEG", quality=quality, optimize=True)
    return True

def compress_folder_to_4mb(folder_path):
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        image_path = os.path.join(folder_path, filename)
        temp_path = os.path.join(folder_path, f"__temp__{filename}")

        # 압축 실행
        success = compress_image_to_under_4mb(image_path, temp_path)

        if success:
            os.remove(image_path)  # 원본 삭제
            os.rename(temp_path, image_path)  # 임시 → 원본으로 교체
            print(f"✅ 압축 및 교체 완료: {filename}")
        else:
            if os.path.exists(temp_path):
                os.remove(temp_path)
            print(f"❌ 압축 실패: {filename}")


In [7]:
compress_folder_to_4mb("0613ciga_test2")

✅ 압축 및 교체 완료: 360_F_311285826_hpnEQj8txKtX5mCl4K7oHH4lkGuRxXvU.jpg
✅ 압축 및 교체 완료: 360_F_473057378_dvyrCRb7ZlOy0E8witgFIF6fkYw8LI0r.jpg
✅ 압축 및 교체 완료: 3df993938ec00a4fa2b5fc545e4d4a7b0975537f.71.9.9.3.jpg
✅ 압축 및 교체 완료: 4429bdc8f4bed76d1d63907a2008bfbd-1536x864.jpg
✅ 압축 및 교체 완료: drain5.jpg
✅ 압축 및 교체 완료: imgdac05e8dzik0zj.jpg
✅ 압축 및 교체 완료: o1080116614786240065.jpg
✅ 압축 및 교체 완료: o2304129614627900845.jpg
✅ 압축 및 교체 완료: OIP (1).jpg
✅ 압축 및 교체 완료: OIP (2).jpg
✅ 압축 및 교체 완료: OIP.jpg
✅ 압축 및 교체 완료: R (1).jpg
✅ 압축 및 교체 완료: R.jpg
✅ 압축 및 교체 완료: tsdr-pl-pipe230618t-5.jpg
